In [3]:
import math

def maxdeflection(clearspan, membertype):
    """
    NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.

    Parameters:
    clearspan: span length (mm)
    membertype:
        'flatroofs'    = flat roof members due to maximum lr and r
        'floor'       = floor members due to l
        'roofdamaged' = roof or floors that are likely to be damaged by large deflections
        'roofundamaged'= roof or floors that are not likely to be damaged by large deflection
    """
    if membertype == 'flatroofs':
        return clearspan / 180
    elif membertype == 'floor':
        return clearspan / 360
    elif membertype == 'roofdamaged':
        return clearspan / 480
    elif membertype == 'roofundamaged':
        return clearspan / 240
    else:
        raise ValueError("Invalid member type. Must be flatroofs, floor, roofdamaged, or roofundamaged")

def momentofinertia(width, height):
    ig = width * math.pow(height, 3) / 12  # in mm^4
    return ig

def modulusofrupture(ig, distance, maximummoment):
    sectionmodulus = ig / distance 
    fr = maximummoment / sectionmodulus
    return fr

def crackingmoment(fr, ig, height):
    """
    NSCP 2015 Eq. 424.2.3.5b. Cracking moment  

    Parameters:
    ig: gross moment of inertia (mm^4)
    fr: modulus of rupture (MPa)
    yt: distance from neutral axis to extreme fiber (mm)
    """
    yt = height / 2
    mcr = (fr * ig) / yt
    return mcr

def crackedmomentofinertia(width, distance, modularratio, asbar, depth):
    icr = width * math.pow(distance, 3) / 3 + modularratio * asbar * math.pow((depth - distance), 2)
    return icr

def effectivemomentofinertia(mcr, ma, ig, icr):
    """
    NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia

    Parameters:
    mcr: cracking moment (n-mm)
    ma: applied moment (n-mm)
    ig: gross moment of inertia (mm^4)
    icr: cracked section moment of inertia (mm^4)
    """
    ratio = math.pow(mcr / ma, 3)
    ie = ratio * ig + (1 - ratio) * icr
    if ie > ig:
        ie = ig
    return ie

def loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload):
    loaddeflectionvalue = 0
    if beamtype == 'simply supported':
        if pointload:
            loaddeflectionvalue += (pointload * clearspan ** 3) / (48 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (5 * uniformload * clearspan ** 4) / (384 * elasticity * ig)
    elif beamtype == 'cantilever':
        if pointload:
            loaddeflectionvalue += (pointload * clearspan ** 3) / (3 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * clearspan ** 4) / (8 * elasticity * ig)
    elif beamtype == 'fixed fixed':
        if pointload:
            loaddeflectionvalue += (pointload * clearspan ** 3) / (192 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * clearspan ** 4) / (384 * elasticity * ig)
    elif beamtype == 'fixed propped':
        if pointload:
            loaddeflectionvalue += (pointload * clearspan ** 3) / (48 * elasticity * ig)
        if uniformload:
            loaddeflectionvalue += (uniformload * clearspan ** 4) / (185 * elasticity * ig)
    else:
        raise ValueError("Invalid beam type specified.")
    return loaddeflectionvalue

def timedependentfactor(sustainedduration):
    """
    NSCP 2015 Table 424.2.4.1.3
    Time-dependent factor for sustained loads.

    Parameters:
    sustainedduration: duration of sustained load in months
                       Valid values: 3, 6, 12, or >=60.
    """
    if sustainedduration == 3:
        timefactor = 1.0
    elif sustainedduration == 6:
        timefactor = 1.2
    elif sustainedduration == 12:
        timefactor = 1.4
    elif sustainedduration >= 60:
        timefactor = 2.0
    else:
        raise ValueError("Invalid sustainedduration. Must be 3, 6, 12, or >=60")
    return timefactor

def timedependentdeflection(deflection, timefactor, compressionbar, width, depth):
    """
    NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
    Additional deflection due to creep and shrinkage is the product of the immediate deflection and a factor lambda_delta.

    Parameters:
    deflection: immediate deflection due to sustained load (mm)
    timefactor: time-dependent factor based on sustained load duration
    compressionbar: diameter of the compression bar (mm)
    width: width of the section (mm)
    depth: depth of the section (mm)
    """
    asprime = math.pi * math.pow(compressionbar / 2, 2)
    rhoprime = asprime / (width * depth)
    lambda_delta = timefactor / (1 + 50 * rhoprime)
    timedependentdeflectionvalue = deflection * lambda_delta
    return timedependentdeflectionvalue

def finaldeflection(maxdeflection, timedependentdeflectionvalue, loaddeflectionvalue):
    """
    Determines the governing deflection as the sum of the time-dependent and load deflections,
    but capped by the maximum permissible deflection.
    """
    totaldeflection = timedependentdeflectionvalue + loaddeflectionvalue
    if totaldeflection <= maxdeflection:
        return totaldeflection
    else:
        return maxdeflection

if __name__ == '__main__':
    clearspan = 6000  # mm
    membertype = 'floor'  # options: 'flatroofs', 'floor', 'roofdamaged', 'roofundamaged'
    width = 200   # mm
    height = 300  # mm
    distance = height / 2  # mm (distance from neutral axis to extreme fiber)
    maximummoment = 1e9  # n-mm
    modularratio = 10  # sample value
    barsize = 16  # mm
    asbar = math.pi * math.pow(barsize / 2, 2)  # mm^2
    ma = 5e8  # n-mm, applied moment
    beamtype = 'simply supported'
    elasticity = 10000  # N/mm^2, sample modulus of elasticity
    pointload = 10000   # N, sample point load
    uniformload = 1  # N/mm, sample uniform load
    sustainedduration = 60  # 3,6,12,>=60 months

    # Maximum permissible deflection
    max_defl = maxdeflection(clearspan, membertype)
    print(f"Maximum permissible deflection for {membertype}: {max_defl:.2f}")

    # Gross moment of inertia calculation
    ig = momentofinertia(width, height)
    print(f"Gross moment of inertia (Ig): {ig:.2f}")

    # Modulus of rupture calculation
    fr = modulusofrupture(ig, distance, maximummoment)
    print(f"Modulus of rupture (fr): {fr:.2f}")

    # Cracking moment calculation using NSCP Eq. 424.2.3.5b
    mcr = crackingmoment(fr, ig, height)
    print(f"Cracking moment (Mcr): {mcr:.2f}")

    # Cracked moment of inertia calculation
    icr = crackedmomentofinertia(width, distance, modularratio, asbar, height)
    print(f"Cracked moment of inertia (Icr): {icr:.2f}")

    # Effective moment of inertia calculation using NSCP Eq. 424.2.3.5a
    ie = effectivemomentofinertia(mcr, ma, ig, icr)
    print(f"Effective moment of inertia (Ie): {ie:.2f}")

    # Load deflection for a simply supported beam
    ld = loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload)
    print(f"Load deflection: {ld:.2f}")

    # Time-dependent factor for sustained loads
    timefactor = timedependentfactor(sustainedduration)
    print(f"Time-dependent factor: {timefactor:.2f}")

    # Time-dependent deflection (creep and shrinkage effects)
    tdd = timedependentdeflection(ld, timefactor, 16, width, height)
    print(f"Time-dependent deflection: {tdd:.2f}")

    # Final deflection
    fd = finaldeflection(max_defl, tdd, ld)
    print(f"Final deflection: {fd:.2f}")

Maximum permissible deflection for floor: 16.67
Gross moment of inertia (Ig): 450000000.00
Modulus of rupture (fr): 333.33
Cracking moment (Mcr): 1000000000.00
Cracked moment of inertia (Icr): 270238934.21
Effective moment of inertia (Ie): 450000000.00
Load deflection: 13.75
Time-dependent factor: 2.00
Time-dependent deflection: 23.55
Final deflection: 16.67


In [2]:
import math

def maxdeflection(clearspan, membertype):
    """
    NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.

    Parameters:
    clearspan: span length (mm)
    membertype:
        'flatroofs'    = flat roof members due to maximum lr and r
        'floor'       = floor members due to l
        'roofdamaged' = roof or floors that are likely to be damaged by large deflections
        'roofundamaged'= roof or floors that are not likely to be damaged by large deflection
    """
    if membertype == 'flatroofs':
        result = clearspan / 180
    elif membertype == 'floor':
        result = clearspan / 360
    elif membertype == 'roofdamaged':
        result = clearspan / 480
    elif membertype == 'roofundamaged':
        result = clearspan / 240
    else:
        raise ValueError("Invalid member type. Must be flatroofs, floor, roofdamaged, or roofundamaged")
    return result

def momentofinertia(width, height):
    result = width * math.pow(height, 3) / 12  # in mm^4
    return result

def modulusofrupture(ig, distance, maximummoment):
    sectionmodulus = ig / distance 
    result = maximummoment / sectionmodulus
    return result

def crackingmoment(fr, ig, height):
    """
    NSCP 2015 Eq. 424.2.3.5b. Cracking moment  

    Parameters:
    ig: gross moment of inertia (mm^4)
    fr: modulus of rupture (MPa)
    yt: distance from neutral axis to extreme fiber (mm)
    """
    yt = height / 2
    result = (fr * ig) / yt
    return result

def crackedmomentofinertia(width, distance, modularratio, asbar, depth):
    result = width * math.pow(distance, 3) / 3 + modularratio * asbar * math.pow((depth - distance), 2)
    return result

def effectivemomentofinertia(mcr, ma, ig, icr):
    """
    NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia

    Parameters:
    mcr: cracking moment (n-mm)
    ma: applied moment (n-mm)
    ig: gross moment of inertia (mm^4)
    icr: cracked section moment of inertia (mm^4)
    """
    ratio = math.pow(mcr / ma, 3)
    ie = ratio * ig + (1 - ratio) * icr
    if ie > ig:
        result = ig
    else:
        result = ie
    return result

def loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload):
    result = 0
    if beamtype == 'simply supported':
        if pointload:
            result += (pointload * clearspan ** 3) / (48 * elasticity * ig)
        if uniformload:
            result += (5 * uniformload * clearspan ** 4) / (384 * elasticity * ig)
    elif beamtype == 'cantilever':
        if pointload:
            result += (pointload * clearspan ** 3) / (3 * elasticity * ig)
        if uniformload:
            result += (uniformload * clearspan ** 4) / (8 * elasticity * ig)
    elif beamtype == 'fixed fixed':
        if pointload:
            result += (pointload * clearspan ** 3) / (192 * elasticity * ig)
        if uniformload:
            result += (uniformload * clearspan ** 4) / (384 * elasticity * ig)
    elif beamtype == 'fixed propped':
        if pointload:
            result += (pointload * clearspan ** 3) / (48 * elasticity * ig)
        if uniformload:
            result += (uniformload * clearspan ** 4) / (185 * elasticity * ig)
    else:
        raise ValueError("Invalid beam type specified.")
    return result

def timedependentfactor(sustainedduration):
    """
    NSCP 2015 Table 424.2.4.1.3
    Time-dependent factor for sustained loads.

    Parameters:
    sustainedduration: duration of sustained load in months
                       Valid values: 3, 6, 12, or >=60.
    """
    if sustainedduration == 3:
        result = 1.0
    elif sustainedduration == 6:
        result = 1.2
    elif sustainedduration == 12:
        result = 1.4
    elif sustainedduration >= 60:
        result = 2.0
    else:
        raise ValueError("Invalid sustainedduration. Must be 3, 6, 12, or >=60")
    return result

def timedependentdeflection(deflection, timefactor, compressionbar, width, depth):
    """
    NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
    Additional deflection due to creep and shrinkage is the product of the immediate deflection and a factor lambda_delta.

    Parameters:
    deflection: immediate deflection due to sustained load (mm)
    timefactor: time-dependent factor based on sustained load duration
    compressionbar: diameter of the compression bar (mm)
    width: width of the section (mm)
    depth: depth of the section (mm)
    """
    asprime = math.pi * math.pow(compressionbar / 2, 2)
    rhoprime = asprime / (width * depth)
    lambda_delta = timefactor / (1 + 50 * rhoprime)
    result = deflection * lambda_delta
    return result

def finaldeflection(maxdeflection, timedependentdeflectionvalue, loaddeflectionvalue):
    """
    Determines the governing deflection as the sum of the time-dependent and load deflections,
    but capped by the maximum permissible deflection.
    """
    totaldeflection = timedependentdeflectionvalue + loaddeflectionvalue
    if totaldeflection <= maxdeflection:
        result = totaldeflection
    else:
        result = maxdeflection
    return result

if __name__ == '__main__':
    clearspan = 6000  # mm
    membertype = 'floor'  # options: 'flatroofs', 'floor', 'roofdamaged', 'roofundamaged'
    width = 200   # mm
    height = 300  # mm
    distance = height / 2  # mm (distance from neutral axis to extreme fiber)
    maximummoment = 1e9  # n-mm
    modularratio = 10  # sample value
    barsize = 16  # mm
    asbar = math.pi * math.pow(barsize / 2, 2)  # mm^2
    ma = 5e8  # n-mm, applied moment
    beamtype = 'simply supported'
    elasticity = 10000  # N/mm^2, sample modulus of elasticity
    pointload = 10000   # N, sample point load
    uniformload = 1  # N/mm, sample uniform load
    sustainedduration = 60  # months

    # Maximum permissible deflection
    max_defl = maxdeflection(clearspan, membertype)
    print(f"Maximum permissible deflection for {membertype}: {max_defl:.2f}")

    # Gross moment of inertia calculation
    ig = momentofinertia(width, height)
    print(f"Gross moment of inertia (Ig): {ig:.2f}")

    # Modulus of rupture calculation
    fr = modulusofrupture(ig, distance, maximummoment)
    print(f"Modulus of rupture (fr): {fr:.2f}")

    # Cracking moment calculation using NSCP Eq. 424.2.3.5b
    mcr = crackingmoment(fr, ig, height)
    print(f"Cracking moment (Mcr): {mcr:.2f}")

    # Cracked moment of inertia calculation
    icr = crackedmomentofinertia(width, distance, modularratio, asbar, height)
    print(f"Cracked moment of inertia (Icr): {icr:.2f}")

    # Effective moment of inertia calculation using NSCP Eq. 424.2.3.5a
    ie = effectivemomentofinertia(mcr, ma, ig, icr)
    print(f"Effective moment of inertia (Ie): {ie:.2f}")

    # Load deflection for a simply supported beam
    ld = loaddeflection(beamtype, clearspan, elasticity, ig, pointload, uniformload)
    print(f"Load deflection: {ld:.2f}")

    # Time-dependent factor for sustained loads
    timefactor = timedependentfactor(sustainedduration)
    print(f"Time-dependent factor: {timefactor:.2f}")

    # Time-dependent deflection (creep and shrinkage effects)
    tdd = timedependentdeflection(ld, timefactor, barsize, width, height)
    print(f"Time-dependent deflection: {tdd:.2f}")

    # Final deflection
    fd = finaldeflection(max_defl, tdd, ld)
    print(f"Final deflection: {fd:.2f}")


Maximum permissible deflection for floor: 16.67
Gross moment of inertia (Ig): 450000000.00
Modulus of rupture (fr): 333.33
Cracking moment (Mcr): 1000000000.00
Cracked moment of inertia (Icr): 270238934.21
Effective moment of inertia (Ie): 450000000.00
Load deflection: 13.75
Time-dependent factor: 2.00
Time-dependent deflection: 23.55
Final deflection: 16.67
